In [ ]:
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load and preprocess data
X = df.drop('fraud_label', axis=1)  # Replace 'fraud_label' with your actual label column
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define autoencoder model
input_dim = X.shape[1]
autoencoder = models.Sequential([
    layers.InputLayer(input_shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=32, validation_split=0.1)

# Predict and detect anomalies
reconstructions = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - reconstructions, 2), axis=1)
threshold = np.percentile(mse, 95)
df['anomaly'] = mse > threshold

# Evaluate detection rate
fraud_detected = df[df['anomaly'] == True]
print(f"Detected {len(fraud_detected)} potential fraud cases.")